In [1]:
using Pkg
Pkg.activate(".")

using Plots, HypertextLiteral, Random
include("../src/load.jl") # load datasets
include("../src/surrogate.jl")
include("../src/run.jl") # run tests

  Activating project at `~/Research/SurrogateDistanceModels/notebooks`
┌ Info: Precompiling SimilaritySearch [053f045d-5466-53fd-b400-a066f88fe02a]
└ @ Base loading.jl:1662


test_searchgraph (generic function with 2 methods)

In [2]:
Threads.nthreads()

64

In [3]:
]status

Status `~/Research/SurrogateDistanceModels/notebooks/Project.toml`
  [336ed68f] CSV v0.10.4
  [a93c6f00] DataFrames v1.3.4
  [0b91fe84] DisplayAs v0.1.6
  [c5bfea45] Embeddings v0.4.2
  [c27321d9] Glob v1.3.0
  [f67ccb44] HDF5 v0.16.11
  [ac1192a8] HypertextLiteral v0.9.4
  [033835bb] JLD2 v0.4.22
  [682c06a0] JSON v0.21.3
⌃ [91a5bcdd] Plots v1.31.5
  [612083be] Queryverse v0.7.0
  [ca7ab67e] SimSearchManifoldLearning v0.2.5
  [053f045d] SimilaritySearch v0.9.4 `../../SimilaritySearch.jl`
  [f3b207a7] StatsPlots v0.15.1
Info Packages marked with ⌃ have new versions available


In [6]:
function run_experiment(D, k;
        kscalelist=[1, 8, 16],
        npairslist=[256, 512, 1024, 2048],
        npoolslist=[32, 64, 128, 256],
        ssizelist=[4, 8, 16],
        topklist=[15, 31, 63],
        npermslist=[4, 8, 16],
        permsizelist=[64]
    )
    D.params["k"] = k
    D.params["enctime"] = 0.0
    Gold = test_exhaustive(nothing, D.db, D.queries, D.dist, copy(D.params), k)
    test_searchgraph(Gold, D.db, D.queries, D.dist, copy(D.params), k)
    test_searchgraph(Gold, D.db, D.queries, D.dist, copy(D.params), k, 0.6)

    surrogates = []
    dim = length(D.db[1])
    for kscale in kscalelist
        # push!(surrogates, BinaryHammingFixedPairs(kscale))
        
        for npairs in npairslist
            push!(surrogates, BinaryHammingSurrogate(kscale, npairs, dim))
        end
        
        for nperms in npermslist, permsize in permsizelist
            push!(surrogates, Perms(permsize, nperms, dim, kscale))
            s = ceil(Int, permsize/3)
            push!(surrogates, BinPerms(s, permsize, nperms, dim, kscale))            
        end
        
        for ssize in ssizelist, npools in npoolslist
            push!(surrogates, MaxHashSurrogate(ssize, npools, dim, kscale))
        end        
        
        #=
        for topk in topklist
            #push!(surrogates, TopKSurrogate(topk, dim, kscale))
            push!(surrogates, SmoothedTopK(topk, dim, kscale))
        end =#
    end

    for E in surrogates
        enctime = @elapsed H = encode(E, D.db, D.queries, copy(D.params))
        H.params["enctime"] = enctime
        test_exhaustive(Gold, H.db, H.queries, H.dist, copy(H.params), k * kscale(E))
        test_searchgraph(Gold, H.db, H.queries, H.dist, copy(H.params), k * kscale(E), 0)
    end
end

run_experiment (generic function with 1 method)

In [ ]:
k=32

let
    D = load_glove_400k()
    @show size(D.db.matrix), D.dist
    run_experiment(D, k)
end

let
    D = load_wit_300k()
    @show size(D.db.matrix), D.dist
    run_experiment(D, k)
end

let
    D = load_glove_1m()
    @show size(D.db.matrix), D.dist
    run_experiment(D, k)
end

let
    D = load_bigann_1m()
    @show size(D.db.matrix), D.dist
    run_experiment(D, k)
end


(size(D.db.matrix), D.dist) = ((100, 390000), NormalizedCosineDistance())
(:Perms, permsize, nperms, dim, kscale) = (:Perms, 64, 4, 100, 1)
(shift, permsize, nperms, dim, kscale) = (22, 64, 4, 100, 1)
(:Perms, permsize, nperms, dim, kscale) = (:Perms, 64, 8, 100, 1)
(shift, permsize, nperms, dim, kscale) = (22, 64, 8, 100, 1)
(:Perms, permsize, nperms, dim, kscale) = (:Perms, 64, 16, 100, 1)
(shift, permsize, nperms, dim, kscale) = (22, 64, 16, 100, 1)
(:Perms, permsize, nperms, dim, kscale) = (:Perms, 64, 4, 100, 8)
(shift, permsize, nperms, dim, kscale) = (22, 64, 4, 100, 8)
(:Perms, permsize, nperms, dim, kscale) = (:Perms, 64, 8, 100, 8)
(shift, permsize, nperms, dim, kscale) = (22, 64, 8, 100, 8)
(:Perms, permsize, nperms, dim, kscale) = (:Perms, 64, 16, 100, 8)
(shift, permsize, nperms, dim, kscale) = (22, 64, 16, 100, 8)
(:Perms, permsize, nperms, dim, kscale) = (:Perms, 64, 4, 100, 16)
(shift, permsize, nperms, dim, kscale) = (22, 64, 4, 100, 16)
(:Perms, permsize, nperms, dim,